# Explanatory notes on Wu et al 2006
Making concrete some of the asides; adding explanation for formulas.

In [10]:
# Imports for a few quick demos below
import matplotlib.pyplot as plt
import numpy as np

# Derivation of normal equations for ML and MAP estimation of regression weights

This notebook describes two probablistic frameworks for estimating $\beta$ weights in linear regressions: The *Maximum Likelihood* (ML) framework from standard probability theory and the *Maximum A Posteriori* (or MAP) framework, from Bayesian probability theory. 

The core idea behind both of these approaches is that you want to find a probable or likely model, and that the first criterion for a likely model is that it does not mess up (i.e., generate large prediction errors). Large errors make a model unlikely.

The MAP framework adds something to this base assumption, which is that model weights ($\beta$ values) that deviate from a normal distribution also make a model unlikely. 

# First, we will tackle Maximum Likelihood

The derivation below is complex, so we will give an overview here first. The steps we will take are: 

1. Create a mathematical expression for the probability of our model, given $\beta$
2. Simplify the probability expression with some mathematical tricks
3. Compute the $\beta$ value that gives the maximally probable model. This involves:
    1. taking the derivative of the probability expression (calculus!)
    2. setting it to zero (the point at which the slope isn't changing is the minimum of the curve). We find the minimum rather than the maximum because some of the simplification we did above invovles a negative sign, which flips the probability curve upside-down.
    3. Solving the equation *[complex probability math involving $\beta$] = 0* for $\beta$

So, the first task for us here will be to express the probability of our models as a function of our data and our model parameters. 

## Terms
* $X$ : characterization of stimulus ($S$ is used in Wu et al, but I prefer X to stick to traditional regression notation)
* $Y$ : responses (the Wu paper uses $r$)
* $\beta$ : parameters associated with columns of X (the paper uses $\theta$, which expresses more generally the parameters of ANY model (not strictly a linear model) that you wish to estimate. I find it clearer to think in terms of the $\beta$ values in an fMRI regression, so I will stick with $\beta$ here.
* $p(Y|X,\beta)$ : the probability of Y (the response) given $X$ and $\beta$.
* $N(\mu, \sigma^2)$ : a normal distribution with some mean (here, $\mu$, though whatever appears first in the parentheses after N should be interpreted as the mean) and variance (here, $\sigma^2$, though whatever appears second in the parentheses - after the comma - should be interpreted as the variance). This expression is commonly used as a substitute for a more complex exponential equation (e.g. $e^-{\frac{(x-\mu_x)^2 ...}{...}}$)
* ${\displaystyle \prod_{n=1}^{N} x_n = x_1 * x_2 * x_3 ... *x_N}$


# Step 1: Express model probability

If we assume that each response in a time series is draw independently from the response distributon, then the probability of all the responses in a time series is a product of the probability of each individual response. Thus:

## $p(\beta | X, Y) = {\displaystyle \prod_{n=1}^{N} N(y_n | X_n\beta, \sigma^2)}$


# Step 2: Simplify the expression
If we expand out the expression for the normal probability distribution, this will get a bit hairy, so we will take a few steps to simplify this expression before expanding it out. The first simplifying step we will take is to take the negative natural logarithm of both sides:

## $-ln(p(\beta | X, Y)) \propto -ln({\displaystyle \prod_{n=1}^{N} N(y_n | X_n\beta, \sigma^2)})$

We can use a property of logarithms to simplify this expression: the log of a product of two or more numbers is the same as the sum of the logarithms of those numbers. Check it out: 

In [72]:
print(-np.log(5*2))
print(-np.log(5) + -np.log(2))

-2.302585092994046
-2.3025850929940455


To be bet even more explicit:

In [77]:
# Define a longer list of numbers
x = np.array([1.3,2,4,6,3,5,2]).astype(np.float)
# This...
print(np.log(np.prod(x)))
# ...is exactly equal to this!
print(np.sum(np.log(x)))
# ... because Math.

7.534762657037538
7.534762657037538


Thus, the $\Pi$ expression above converst to a $\Sigma$ expression:
## $-ln(p(\beta | X, Y)) \propto {\displaystyle \sum_{n=1}^{N} ln(N(y_n | x_n\beta, \sigma^2))}$

Now, we will expand the expression of the Gaussian probability function out (to an exponential). Note the formulation here: $X_n\beta$ describes the mean of the Gaussian distribution, the probability of which is evaluated at $y_n$, with some underlying $\sigma^2$ width (which as we will see below doesn't end up mattering much).

## $-ln(p(\beta | X, Y)) \propto {\displaystyle \sum_{n=1}^{N} ln \bigg(\frac{1}{\sqrt{2\pi\sigma^2}}e^{-\frac{(y_n-X_n\beta)^2}{2\sigma^2}}\bigg)}$

Replace the square root with a fractional exponent (same same...)

## $-ln(p(\beta | X, Y)) \propto {\displaystyle \sum_{n=1}^{N} ln \bigg((2\pi\sigma^2)^{-\frac{1}{2}}e^{-\frac{(y_n-X_n\beta)^2}{2\sigma^2}}\bigg)}$

Distribute the logarithm to simplify further (take ln of the fraction and the exponential separately, and remember that the log of a product is the same as the sum of the logs, and that $ln(a^b) = b * ln(a)$:

## $-ln(p(\beta | X, Y)) \propto {\displaystyle -\frac{N}{2}ln(\sigma^2) - \frac{N}{2}ln(2\pi) + \frac{1}{\sigma^2} \bigg({\frac{1}{2} \sum_{n=1}^{N}(y_n-X_n\beta)^2}\bigg)}$

# Step 3A: Take the derivative of the simplified expression
Since we will be taking the derivative of this statement with respect to $\beta$, all the constant terms *not* relating to $\beta$ (everything in gray below) won't matter and can be dropped:


## $-ln(p(\beta | X, Y)) \propto $ <span style="color:gray">$ {\displaystyle-\frac{N}{2}ln(\sigma^2) - \frac{N}{2}ln(2\pi) + \frac{1}{\sigma^2} }$ </span> ${\displaystyle \bigg({\frac{1}{2} \sum_{n=1}^{N}(y_n-X_n\beta)^2}\bigg)}$

So we are left with: 

##  $-ln(p(\beta | X, Y)) \propto {\displaystyle {\frac{1}{2} \sum_{n=1}^{N}(y_n-X_n\beta)^2}}$

This expression is a parabola. We want to fnd its minimum, so we differentiate with respect to $\beta$  set equal to 0, and solve for $\beta$. (Remember, the minimum of a curve is where the slope - the differential - is equal to zero) ***This is the only step of calculus in this whole derivation!***

## ${\displaystyle \frac{\delta}{\delta\beta}\bigg({\frac{1}{2} \sum_{n=1}^{N}(y_n-X_n\beta)^2}\bigg) = \sum_{n=1}^{N}X^T(y_n-X_n\beta)}$

Convert to matrix form for simplicity:

## $X^TY-X^TX\beta = 0$

# Step 3C: Solve for $\beta$ 

Add $X^TX\beta$ to both sides:

## $X^TX\beta = X^TY$

And then multiply both sides by the inverse of $X^TX$, because $(X^TX)^{-1}(X^TX) = I$, where $I$ is the identity matrix (the matrix equivalent of multiplying by 1:

## $(X^TX)^{-1}X^TX\beta = (X^TX)^{-1}X^TY$

In the end here, we get the normal equations for computing $\beta$ under the maximum likelihood framework (i.e., NO PRIOR ON THE WEIGHTS YET):
## $\beta = (X^TX)^{-1}X^TY$

This equivalent to computing the ordinary least squares estimate of $\beta$!

# Now, let's compute the MAP estimate of the $\beta$ weights.
This method of estimating regression weights takes into account a PRIOR on the size of the beta weights. Meaning, a model is considered less probable if its weights don't follow a normal distribution. (The main thing the $\beta$ weight prior does is make a model with a few very large weights less likely, which penalizes overfitting!)

# Step 1: Express model probability
Here, we make use of Bayes rule to compute model probability. Bayes rule says: 

## $p(a, b) = p(b, a)$

## $p(a, b)  = p(a\ |\ b)\ p(b)$

## $p(b, a) = p(b\ |\ a)\ p(a)$

## $p(a\ |\ b) = \frac{p(b\ |\ a) p(a)}{p(b)}$

For our purposes, we express the probability of the model parameters ($\beta$) given the probability of the data given the model + parameters and the prior probability of the $\beta$ values

## $p(\beta\ |\ X, Y) = \frac{p(Y\ |\ X,\beta)\ p(\beta)}{p(Y)}$

We express this as a likelihood, because who knows what the probability of our data is, and likelihood is sufficient for our purposes. The first term here is the probability of our data given the model (which will be a distribution that expresses that models that make poor predictions are unlikely), and the second term is the prior probability of our model parameters (our assumption about the weights, i.e. that there shouldn't be a few big ones, that the values of the weights should be normally distributed):

## $p(\beta\ |\ X,Y) \propto p(Y\ |\ X,\beta)\ p(\beta)$

We express $p(\beta)$ as a normal distribution, with mean 0 and standard deviation $\lambda$ (which is now a free parameter we are introducing!)

## $p(\beta\ |\ \lambda) = N(\beta\ |\ 0, \lambda^{-1}I)$

Expanded out to the full exponential expression, that would be:

## $p(\beta\ |\ \lambda) = \frac{1}{\sqrt{2\pi\sigma^2}}\ e^{-\frac{(\beta-0)^2}{2\lambda}}$

So the overall expression takes the form of a multiplication of normal distributions:

## $p(\beta\ |\ X, Y) \propto {\displaystyle \bigg(\prod_{n=1}^{N} N(y_n\ |\ X_n\beta, \sigma^2)\bigg)\ N(\beta\ |\ 0, \lambda^{-1}I)}$

# Step 2: Simplify the expression
... by taking the negative log of both sides.

## $-ln(p(\beta\ |\ X, Y)) \propto {\displaystyle {\frac{1}{2} \sum_{n=1}^{N}(y_n-X_n\beta)^2} + \frac{\lambda}{2}\beta^2}$

Note that we skipped several steps here that we did explicitly in the derivation above. Remember that log of multiplications convert to addition of logs, and for both parts of the expression we drop a bunch of terms that won't affect the location of the minimum. It's a good exercise to see if you can do the derivative of the exponential expression for the prior!

# Step 3: Take the derivative of the simplified expression, set to zero, and solve for $\beta$
... and set it to zero to find the minimum negative log likelihood, a.k.a. maximum posterior probability
## ${\displaystyle \frac{\delta}{\delta\beta}\bigg(\frac{1}{2} \sum_{n=1}^{N}X(y_n-X_n\beta)^2 + \frac{\lambda}{2}\beta^2\bigg)} = 0$

Convert result to matrix form:

## $X^TY-X^TX\beta + \lambda\beta = 0$

... and solve for $\beta$:

## $X^TX\beta + \lambda\beta = X^TY$

## $(X^TX + \lambda I)\beta = X^TY$

# AND LO: the normal equation for ridge regression
## $\beta = (X^TX + \lambda I)^{-1}X^TY$


# TA-DAAAA!